# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [1]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [2]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [3]:
earthquake_json['metadata']

{'generated': 1718208992000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2024-05-12&endtime=2024-06-11',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.14.1',
 'count': 10262}

Each element in the JSON array `features` is a row of data for our dataframe.

In [4]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [5]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.75,
  'place': '12 km SE of Hanksville, Utah',
  'time': 1718063955360,
  'updated': 1718114141520,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/uu80072966',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=uu80072966&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 47,
  'net': 'uu',
  'code': '80072966',
  'ids': ',uu80072966,',
  'sources': ',uu,',
  'types': ',origin,phase-data,',
  'nst': 10,
  'dmin': 0.3829,
  'rms': 0.09,
  'gap': 122,
  'magType': 'md',
  'type': 'earthquake',
  'title': 'M 1.8 - 12 km SE of Hanksville, Utah'},
 'geometry': {'type': 'Point',
  'coordinates': [-110.598, 38.3043333333333, 6.04]},
 'id': 'uu80072966'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [6]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.75,"12 km SE of Hanksville, Utah",1718063955360,1718114141520,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",uu80072966,",",uu,",",origin,phase-data,",10.0,0.382900,0.09,122.0,md,earthquake,"M 1.8 - 12 km SE of Hanksville, Utah"
1,1.80,"48 km SSW of Cantwell, Alaska",1718063825687,1718063941523,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak0247gb3f60,",",ak,",",origin,phase-data,",NaN,NaN,0.76,NaN,ml,earthquake,"M 1.8 - 48 km SSW of Cantwell, Alaska"
2,1.48,"6 km WNW of Cobb, CA",1718063617020,1718063712627,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc75020542,",",nc,",",nearby-cities,origin,phase-data,",16.0,0.011920,0.05,72.0,md,earthquake,"M 1.5 - 6 km WNW of Cobb, CA"
3,2.14,"7 km SW of Volcano, Hawaii",1718063529200,1718065342270,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",hv74271791,",",hv,",",origin,phase-data,",32.0,0.004276,0.10,42.0,ml,earthquake,"M 2.1 - 7 km SW of Volcano, Hawaii"
4,1.10,"5 km NNW of Boron, CA",1718063231280,1718114331411,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci40619207,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",27.0,0.113300,0.14,46.0,ml,quarry blast,"M 1.1 Quarry Blast - 5 km NNW of Boron, CA"


### (Optional) Write Data to CSV

In [7]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>